In [1]:
from IPython.display import display, HTML
import pandas as pd
import os
import glob

Helper functions.

In [2]:
def get_name_parts(pred_file):
    return os.path.dirname(pred_file).split(os.path.sep)

def get_exp_name(pred_file):
    parts = get_name_parts(pred_file)
#     experiment_name = parts[-3]
    from_index = parts.index('experiments') + 1
    experiment_name = "/".join(parts[from_index:])
    return experiment_name

def get_dataset(pred_file):
    parts = os.path.dirname(pred_file).split(os.path.sep)
    dataset = '_'.join(parts[-2].split('_')[:2])
    return dataset

def calculate_accuracy(pred_file):
    df = pd.read_csv(pred_file)
    df['correct'] = df['label'] == df['predicted']
    accuracy = df['correct'].sum() / len(df['correct'])
    return accuracy

Calculate validation accuracy using the experiment's prediction.csv files.

In [3]:
pred_files = glob.glob("../experiments/**/*prediction*.csv", recursive=True)

summary_df = pd.DataFrame(index=None, columns=['dataset', 'val_accuracy', 'experiment'])

for pred_file in pred_files:
    dataset = get_dataset(pred_file)
    exp_name = get_exp_name(pred_file)
    acc = calculate_accuracy(pred_file)
    summary_df = summary_df.append({'dataset': dataset, 'experiment': exp_name, 'val_accuracy': acc}, 
                      ignore_index=True)

In [4]:
summary_df = summary_df[summary_df.experiment.str.contains("_archived") == False]
sorted_df = summary_df.sort_values(['val_accuracy'], ascending=False).sort_values(['dataset'], ascending=False, kind='mergesort')

pd.set_option('display.max_colwidth', -1)
display(HTML(sorted_df.to_html(index=False)))

dataset,val_accuracy,experiment
pro-est_vs,0.895735,data_augmentation/runtime/add_color_jitter_augment/pro-est_vs_met-die_11/resnet_transfer-101-finetune
multistage_experiment,0.781991,multistage_experiment/BEST_diestrus-colorjitter_transfer-4-to-3
die_vs,0.914692,data_augmentation/runtime/add_color_jitter_augment/die_vs_all_11/resnet_transfer-101-finetune
die_vs,0.909953,data_augmentation/runtime/random_affine_crop/die_vs_all_11/resnet_transfer-101-finetune
die_vs,0.909953,data_augmentation/runtime/add_color_jitter_augment/die_vs_all_11/resnet_transfer-152-finetune
die_vs,0.878571,add_exp11a_images/die_vs_all_11+11a_filtered/resnet_transfer-101-finetune
die_vs,0.796209,model_architecture/num_layers_redo/die_vs_all_11/resnet_transfer-18-finetune
4_class,0.791469,data_augmentation/runtime/random_affine_crop/4_class_11/resnet_transfer-101-finetune
4_class,0.781991,data_augmentation/runtime/add_color_jitter_augment_2/4_class_11/resnet_transfer-101-finetune
4_class,0.781991,multistage_experiment/4_class_11/diestrus-colorjitter_transfer-4-to-3


In [5]:
sorted_df.to_csv('accumulated_accuracies.csv', index=False)

For the user: Filter the above table for a specific experiment search string.

In [12]:
search_string = "11a"
filtered = sorted_df[sorted_df.experiment.str.contains(search_string) == True]
display(HTML(filtered.to_html(index=False)))

dataset,val_accuracy,experiment
die_vs,0.878571,add_exp11a_images/die_vs_all_11+11a_filtered/resnet_transfer-101-finetune
4_class,0.750000,add_exp11a_images/4_class_11+11a_filtered/resnet_transfer-101-finetune
4_class,0.733119,add_exp11a_images/4_class_11+11a/resnet_transfer-101-finetune
